In [1]:
# libraries from finetuning_parameters.py
from finetuning_parameters import get_args
from future.baseline_trainer import BaselineTuner
from future.modules import ptl2classes
from future.hooks import EvaluationRecorder

from data_loader.wrap_sampler import wrap_sampler
import data_loader.task_configs as task_configs
import data_loader.data_configs as data_configs
from future.collocate_fns import task2collocate_fn

import utils.checkpoint as checkpoint
import utils.logging as logging

import torch
import random
import os

# libraries from future/base.py
from torch.utils.data import SequentialSampler, RandomSampler
from future.hooks import EvaluationRecorder
import utils.eval_meters as eval_meters
from seqeval.metrics import f1_score as f1_score_tagging
import torch

# libraries from future/baseline_trainer.py
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy
from future.base import BaseTrainer
from future.hooks.base_hook import HookContainer
from future.hooks import EvaluationRecorder
from torch.utils.data import RandomSampler
from collections import defaultdict, Counter
from tqdm import tqdm

# and so on..
from finetuning_baseline import init_config, init_task, init_hooks

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

2022-02-27 16:25:10.006845: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/ssl/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib64:/usr/include/x86_64-linux-gnu
2022-02-27 16:25:10.006871: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# define task and model (same in finetuning_parameters.py)

In [3]:
parser = get_args()
conf = parser.parse_args(args=[])

In [4]:
conf.dataset_name = 'pawsx'
conf.trn_languages = 'english'
conf.eval_languages = 'english'
conf.finetune_epochs = 10
conf.finetune_batch_size = 256
conf.eval_every_batch = 50
conf.override = False
conf.train_fast = False
conf.world = '0'
conf.finetune_lr = 1e-5

In [5]:
init_config(conf)
model, tokenizer, data_iter, metric_name, collocate_batch_fn = init_task(conf)
adapt_loaders = {}
for language, language_dataset in data_iter.items():
    # NOTE: the sample dataset are refered
    adapt_loaders[language] = wrap_sampler(
        trn_batch_size=conf.finetune_batch_size,
        infer_batch_size=conf.inference_batch_size,
        language=language,
        language_dataset=language_dataset,
    )
hooks = init_hooks(conf, metric_name)

./checkpoint_baseline/pawsx/debug/1645979111_model_task-pawsx_flr-1.0E-05_ftbs-256_ftepcs-10_sd-3_trnfast-False_evalevery-50_tlang-en_vlang-en


parameters: 
	override	False
	experiment	debug
	ptl	bert
	model	bert-base-uncased
	dataset_name	pawsx
	max_seq_len	128
	trn_languages	['english']
	eval_languages	['english']
	finetune_epochs	10
	eval_every_batch	50
	finetune_batch_size	256
	finetune_lr	1e-05
	inference_batch_size	512
	data_path	./data/download
	checkpoint	./checkpoint_baseline
	manual_seed	3
	summary_freq	100
	time_stamp	None
	train_fast	False
	track_time	True
	world	[0]
	is_finished	False
	task	pawsx
	n_sub_process	1
	time_stamp_	1645979111_model_task-pawsx_flr-1.0E-05_ftbs-256_ftepcs-10_sd-3_trnfast-False_evalevery-50_tlang-en_vlang-en
	checkpoint_root	./checkpoint_baseline/pawsx/debug/1645979111_model_task-pawsx_flr-1.0E-05_ftbs-256_ftepcs-10_sd-3_trnfast-False_evalevery-50_tlang-en_vlang-en


experiment platform: workspace-s39hzlgtvbvt-0.
[INFO]: skip de trn: not such file

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

[INFO] loading cached dataset.
[INFO] loading cached dataset.
[INFO] loading cached dataset.


In [6]:
trainer = BaselineTuner(
        conf, collocate_batch_fn=collocate_batch_fn, logger=conf.logger
    )

2022-02-27 16:25:20	Init trainer.


In [7]:
labels = np.empty((0,))
features = np.empty((0, 768))

# trainer.train
opt, model = trainer._init_model_opt(model)
trainer.model = model
trainer.model.eval()

for epoch_index in tqdm(range(1, 1 + 1)):
    trn_iters = []
    for languge in trainer.conf.trn_languages:
        egs = adapt_loaders[language].trn_egs
        assert isinstance(egs.sampler, RandomSampler)
        trn_iters.append(iter(egs))
        
    batches_per_epoch = max(len(ti) for ti in trn_iters)
    for batch_index in range(1, batches_per_epoch + 1):
        trn_loss = []
        for ti in trn_iters:
            try:
                batched = next(ti)
            except StopIteration:
                continue
            batched, golds, uids, _golds_tagging = trainer.collocate_batch_fn(
                batched
            )
            with torch.no_grad():
                hidden = trainer.model.get_last_hidden(**batched)
                labels = np.concatenate((labels, golds.cpu()))
                features = np.concatenate((features, hidden.cpu()), axis=0)
                print (hidden.size())

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

model = model.cuda()


  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size([256, 768])
torch.Size(

100%|██████████| 1/1 [02:23<00:00, 143.92s/it]

torch.Size([249, 768])


In [34]:
import pickle
def save_pickle(file, data):
    with open(file, 'wb') as f:
        pickle.dump(data, f)

In [35]:
base_means = []
base_covs = []
output_dict = {}

for i in np.unique(labels):
    feature = features[labels == i]
    mean = np.mean(feature, axis=0)
    cov = np.cov(feature.T)
    base_means.append(mean)
    base_covs.append(cov)
    
    output_dict[str(int(i))] = [{"mean": mean, "cov": cov}]

In [36]:
save_pickle('features.pkl', output_dict)